In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import torch

In [ ]:
image_dir = os.path.expanduser("~/dataset/dbl/AerialPhotography")
output_path = os.path.expanduser("~/dataset/dbl/parsed_from_exif-sift_matching")
prior_poses = np.load(os.path.join(output_path, "parsed.npy"), allow_pickle=True).item()  # produced by `dji2pose.ipynb`
prior_poses.keys()

In [ ]:
# conver arg list to command
import shlex
def print_command(args: list, return_str: bool = False):
    escaped_args = []
    for i in args:
        escaped_args.append(shlex.quote(i))
    connector = " "
    if len(args) > 3:
        connector = " \\\n    "

    command_str = connector.join(escaped_args)
    if return_str:
        return command_str
    print(command_str)

def get_command(args: list) -> str:
    return print_command(args, True)

# 1. Distance based filter

In [ ]:
camera_centers = prior_poses["c2w"][:, :3, 3]
camera_centers.shape, prior_poses["c2w"][0], camera_centers[0]

In [ ]:
(camera_centers[:, None, :] - camera_centers[None, :, :]).shape

In [ ]:
camera_distances = np.sqrt(np.sum(np.power(camera_centers[:, None, :] - camera_centers[None, :, :], 2), axis=-1))
camera_distances.shape, camera_distances[0]

In [ ]:
camera_distances[:16, :16]

In [ ]:
# make sure the calculation is correct
for i in range(16):
    for j in range(16):
        assert np.isclose(np.linalg.norm(camera_centers[i] - camera_centers[j]), camera_distances[i, j]), (i, j)

In [ ]:
max_distance = 384.

distance_filter_mask = camera_distances < max_distance
distance_filter_image_count = distance_filter_mask.sum(axis=-1)
distance_filter_image_count, \
    distance_filter_image_count.mean().astype(np.int32), \
    distance_filter_image_count.min(), \
    distance_filter_image_count.max()

# 2. View direction based filter

In [ ]:
# Z+ is the view direction
camera_view_directions = prior_poses["c2w"][:, :3, 2]
camera_view_direction_cosine = np.sum(camera_view_directions[:, None, :] * camera_view_directions[None, :, :], axis=-1)
camera_view_direction_cosine[:16, :16]

In [ ]:
camera_view_direction_cosine.min(), camera_view_direction_cosine.max()

## 2.1 Simple result validation
Ignore this section when using other dataset

In [ ]:
image_names = prior_poses["image_name_list"]

In [ ]:
a_i = 5
image_names[a_i]

Pick another image in opposite direction

In [ ]:
b_i = None
for idx, i in enumerate(image_names):
    if i == image_names[a_i].replace("h", "q"):
        b_i = idx
        break
image_names[b_i]

In [ ]:
# < 0. is expected
camera_view_direction_cosine[a_i, b_i]

The direction of the `x` set should be closed to -z

In [ ]:
z_set_image_idx = 5006
image_names[z_set_image_idx]

In [ ]:
# the z close to -Z is expected
camera_view_directions[z_set_image_idx]

The cosine value between `x` set and others should > 0

In [ ]:
for idx, i in enumerate(image_names):
    if os.path.basename(i).endswith(image_names[z_set_image_idx][image_names[z_set_image_idx].rfind("x")+1:]):
        print("{} -> {}".format(i, camera_view_direction_cosine[z_set_image_idx, idx]))

In [ ]:
# min < 0 may appear, but the mean should be large
camera_view_direction_cosine[z_set_image_idx].max(), \
    camera_view_direction_cosine[z_set_image_idx].min(), \
    camera_view_direction_cosine[z_set_image_idx].mean()

In [ ]:
# > 0 and < 1 is expected
camera_view_direction_cosine[image_names.index("h/22h01347.JPG"), image_names.index("z/22z01346.JPG")]

# 3. Z distances

In [ ]:
w2c = np.linalg.inv(prior_poses["c2w"])
np.concatenate([prior_poses["c2w"][0], w2c[0]], axis=1)

In [ ]:
camera_z_distances = np.sum(w2c[:, 2, :3][:, None, :] * camera_centers[None, :, :], axis=-1) + w2c[:, 2, 3][:, None]
camera_z_distances[:16, :16]

## 3.1 Simple result validation
Ignore this section when using other dataset

In [ ]:
# 1346 and 1345 is in front of 1347, while 1348 behind it
# [> 0., > 0., > 0., < 0.]
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("h/22h01346.JPG")], \
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("z/22z01346.JPG")], \
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("h/22h01345.JPG")], \
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("h/22h01348.JPG")]

In [ ]:
# < 0.
camera_z_distances[image_names.index("h/22h01346.JPG"), image_names.index("h/22h01347.JPG")]

In [ ]:
# same position, close to 0.
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("q/22q01347.JPG")], \
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("x/22x01347.JPG")], \
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("y/22y01347.JPG")], \
camera_z_distances[image_names.index("h/22h01347.JPG"), image_names.index("z/22z01347.JPG")]

In [ ]:
# in front of the camera
camera_z_distances_mask = camera_z_distances > 0.
camera_z_distances_mask.sum(axis=-1)

# 4. Build feature matching mask

In [ ]:
mask = np.copy(distance_filter_mask)
mask.sum(axis=-1).min()

Filter out those behind cameras, and do not have similar view direction

In [ ]:
min_cosine_value = np.cos(np.pi / 3)
min_cosine_value

In [ ]:
is_behind_camera = camera_z_distances < 0.
is_not_similar_direction = camera_view_direction_cosine < min_cosine_value
behind_camera_filter_mask = np.logical_and(is_behind_camera, is_not_similar_direction)
mask = np.logical_and(mask, np.logical_not(behind_camera_filter_mask))  # true represents the valid pair
final_image_count = mask.sum(axis=-1)
final_image_count.mean().astype(np.int32), \
final_image_count.min(), \
final_image_count.max()

# 4.1 Simple result validation
Ignore this section when using other dataset

neighbourhoods, same view direction, should always be selected

In [ ]:
for i in [45, 46]:
    print(mask[image_names.index("h/22h01347.JPG"), image_names.index("h/22h013{}.JPG".format(i))])
    print(mask[image_names.index("h/22h013{}.JPG".format(i)), image_names.index("h/22h01347.JPG".format(i))])

In [ ]:
for i in [45, 46, 48, 49]:
    print(mask[image_names.index("x/22x01347.JPG"), image_names.index("x/22x013{}.JPG".format(i))])

neighbourhoods, behind, opposite direction, should not be selected

In [ ]:
for i in [48, 49]:
    print(mask[image_names.index("h/22h01347.JPG"), image_names.index("q/22q013{}.JPG".format(i))])
    print(mask[image_names.index("q/22q013{}.JPG".format(i)), image_names.index("h/22h01347.JPG".format(i))])

neighbourhoods, front, opposite direction, should be selected

In [ ]:
for i in [48, 49]:
    print(mask[image_names.index("q/22q01347.JPG"), image_names.index("h/22h013{}.JPG".format(i))])
    print(mask[image_names.index("h/22h013{}.JPG".format(i)), image_names.index("q/22q01347.JPG".format(i))])

# 5. Build pairs

In [ ]:
from hloc import (
    extract_features,
    match_features,
)

In [ ]:
sfm_pairs = os.path.join(output_path, "pairs-netvlad.txt")
sfm_dir = os.path.join(output_path, "sfm_superpoint+superglue")
os.makedirs(output_path, exist_ok=True)
sfm_pairs, sfm_dir

In [ ]:
retrieval_conf = extract_features.confs["netvlad"]
feature_conf = extract_features.confs["superpoint_aachen"]
matcher_conf = match_features.confs["superglue"]

## 5.1. NetVLAD

In [ ]:
# It takes hours
retrieval_path = extract_features.main(retrieval_conf, image_dir, output_path)
retrieval_path

In [ ]:
from hloc.pairs_from_retrieval import get_descriptors
query_desc = get_descriptors(image_names, retrieval_path)
query_desc.shape

In [ ]:
sim = torch.einsum("id,jd->ij", query_desc, query_desc)
sim.shape

In [ ]:
from hloc.pairs_from_retrieval import pairs_from_score_matrix
num_matched = 64
pairs = pairs_from_score_matrix(
    torch.clone(sim),
    np.logical_not(np.logical_and(
        np.logical_not(np.eye(mask.shape[0], dtype=np.bool_)),  # avoid self pairing
        mask,
    )),
    num_matched,
    min_score=0.25,
)
# pairs = pairs_from_score_matrix(torch.clone(sim), torch.eye(sim.shape[0], dtype=torch.bool).numpy(), num_matched, min_score=0.25)
pairs = [(image_names[i], image_names[j]) for i, j in pairs]
pairs, len(pairs)

In [ ]:
with open(sfm_pairs, "w") as f:
    f.write("\n".join(" ".join([i, j]) for i, j in pairs))
sfm_pairs

# 6. Feature matching
Assuming all the images and cameras have been imported by `dji2pose.ipynb`

In [ ]:
colmap_db_path = os.path.join(output_path, "colmap.db")

## 6.1. [Option 1] SIFT (Recommeneded)

6.1.1. Extract features

In [ ]:
print_command([
    "colmap",
    "feature_extractor",
    "--image_path={}".format(image_dir),
    "--database_path={}".format(colmap_db_path),
])

6.1.2. Match features

In [ ]:
print_command([
    "colmap",
    "matches_importer",
    "--database_path={}".format(colmap_db_path),
    "--match_list_path={}".format(sfm_pairs),
    "--match_type=pairs",
])

## 6.2. [Option 2] SuperPoint + SuperGlue

6.2.1. Extract features

In [ ]:
feature_path = extract_features.main(feature_conf, image_dir, output_path)
feature_path

6.2.2. Match features

In [ ]:
# It takes hours
# LightGlue should be faster a little bit
from pathlib import Path
match_path = match_features.main(
    matcher_conf, Path(sfm_pairs), feature_conf["output"], output_path
)
match_path

6.2.3. Import to colmap database

In [ ]:
from hloc.reconstruction import (
    import_features,
    import_matches,
    estimation_and_geometric_verification,
)

In [ ]:
# map `image_name` to colmap `image_id`
import sqlite3
db = sqlite3.connect(colmap_db_path)
try:
    image_ids = {}
    for name, image_id in db.execute("SELECT name, image_id FROM images;"):
        image_ids[name] = image_id
finally:
    db.close()
image_ids

In [ ]:
# Clear matchs, keypoints and two view geometries
# db = sqlite3.connect(colmap_db_path)
# try:
#     db.execute("DELETE FROM `matches`")
#     db.execute("DELETE FROM `keypoints`")
#     db.execute("DELETE FROM `two_view_geometries`")
#     db.commit()
# finally:
#     db.close()

In [ ]:
import_features(image_ids, colmap_db_path, feature_path)

In [ ]:
import_matches(
    image_ids,
    colmap_db_path,
    sfm_pairs,
    match_path,
    None,
    False,
)

In [ ]:
estimation_and_geometric_verification(colmap_db_path, sfm_pairs, True)

# 7. Mapping

## 7.1. [Option 1] colmap hierarchical_mapper
First choice

In [ ]:
sparse_hierarchical_mapper_dir = os.path.join(output_path, "sparse-hierarchical_mapper")
os.makedirs(sparse_hierarchical_mapper_dir, exist_ok=True)
print(" \\\n    ".join([
    "colmap",
    "hierarchical_mapper",
    "--database_path={}".format(colmap_db_path),
    "--image_path={}".format(image_dir),
    "--output_path={}".format(sparse_hierarchical_mapper_dir),
    "--image_overlap=100",
    "--leaf_max_num_images=900",
    "--Mapper.ba_use_gpu=1",
]))

In [ ]:
# Run a `point_triangulator` is recommended
largest_sparse_model_dir = os.path.join(sparse_hierarchical_mapper_dir, "0")  # Remember to change this
largest_sparse_model_retri_output_dir = "{}-retriangulated".format(largest_sparse_model_dir)
os.makedirs(largest_sparse_model_retri_output_dir, exist_ok=True)
print(" \\\n    ".join([
    "colmap",
    "point_triangulator",
    "--database_path={}".format(colmap_db_path),
    "--image_path={}".format(image_dir),
    "--input_path={}".format(largest_sparse_model_dir),
    "--output_path={}".format(largest_sparse_model_retri_output_dir),
    "--Mapper.ba_use_gpu=1",
]))

In [ ]:
# Run a `bundle_adjuster` is recommended, but outliers should be removed first.
# Take a look `gps_based_sfm_outlier_detection.ipynb`
largest_sparse_model_retri_ba_output_dir = "{}-bundle_adjusted".format(largest_sparse_model_retri_output_dir)
os.makedirs(largest_sparse_model_retri_ba_output_dir, exist_ok=True)
print_command([
    "colmap",
    "bundle_adjuster",
    "--input_path={}".format(largest_sparse_model_retri_output_dir),
    "--output_path={}".format(largest_sparse_model_retri_ba_output_dir),
    "--BundleAdjustment.use_gpu=1",
])

## 7.2. [Option 2] Custom partitioning
Experimental

### 7.2.1. Partitioning

In [ ]:
from internal.utils.partitioning_utils import SceneConfig, PartitionableScene
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
camera_centers = prior_poses["c2w"][:, :3, 3]
reoriented_camera_centers = torch.from_numpy(camera_centers)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal', adjustable='box')
scene_size = torch.max(reoriented_camera_centers, dim=0).values - torch.min(reoriented_camera_centers, dim=0).values
ax.set_xlim([torch.min(reoriented_camera_centers[:, 0]) - 0.1 * scene_size[0], torch.max(reoriented_camera_centers[:, 0]) + 0.1 * scene_size[0]])
ax.set_ylim([torch.min(reoriented_camera_centers[:, 1]) - 0.1 * scene_size[1], torch.max(reoriented_camera_centers[:, 1]) + 0.1 * scene_size[1]])
ax.scatter(reoriented_camera_centers[:, 0], reoriented_camera_centers[:, 1], s=0.2, c="red")
plt.show()

In [ ]:
scene_config = SceneConfig(
    origin=torch.tensor([0., 0.]),
    partition_size=500.,
)
scene = PartitionableScene(scene_config, reoriented_camera_centers[..., :2])
scene.get_bounding_box_by_camera_centers()
scene.get_scene_bounding_box()
scene.plot(scene.plot_scene_bounding_box)
scene.build_partition_coordinates()
scene.plot(scene.plot_partitions)


In [ ]:
scene_config.location_based_enlarge = 0.
assignment_without_enlarging = scene.camera_center_based_partition_assignment()
assignment_without_enlarging.sum(-1).reshape(scene.scene_bounding_box.n_partitions.tolist()[::-1])

In [ ]:
# merge partition with image number less than `min_images` into its most visible neighbour
min_images = 255

assignment_without_enlarging_2d = assignment_without_enlarging.reshape((scene.scene_bounding_box.n_partitions.tolist()[::-1] + [-1]))

assignment_merged = torch.clone(assignment_without_enlarging_2d)

def get_number_of_images(x, y) -> int:
    if x < 0:
        return 0
    if y < 0:
        return 0
    if x >= scene.scene_bounding_box.n_partitions[0]:
        return 0
    if y >= scene.scene_bounding_box.n_partitions[1]:
        return 0
    return assignment_merged[y, x].sum().item()

for y in range(scene.scene_bounding_box.n_partitions[1]):
    for x in range(scene.scene_bounding_box.n_partitions[0]):
        n_images = get_number_of_images(x, y)
        if n_images == 0:
            continue
        if n_images > min_images:
            continue
            
        partition_camera_mean_center = camera_centers[assignment_merged[y, x]].mean(axis=0)
            
        max_neighbour_n_visible = -1
        merge_to_neighbour = None
        
        for neighbour in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
            neighbour_n_images = get_number_of_images(*neighbour)
            if neighbour_n_images == 0:
                continue
                
            partition_image_visible_masks = torch.from_numpy(mask[assignment_merged[y, x].numpy()])  # [N_local_images, N_images]
            partition_neighbour_image_visible_mask = partition_image_visible_masks[:, assignment_without_enlarging_2d[neighbour[1], neighbour[0]]]  # [N_local_images, N_neighbour_partition_images]
            n_visible_neighbour = partition_neighbour_image_visible_mask.sum()
            
            if n_visible_neighbour > max_neighbour_n_visible:
                max_neighbour_n_visible = n_visible_neighbour
                merge_to_neighbour = neighbour
                
            # neighbour_distance = np.linalg.norm(camera_centers[assignment_without_enlarging_2d[y, x]] - partition_camera_mean_center[None, :], axis=-1).min()
            # print(neighbour, neighbour_distance)
            #     
            # if neighbour_distance < min_neighbour_distance:
            #     min_neighbour_n_images = neighbour_n_images
            #     min_neighbour_distance = neighbour_distance
            #     min_neighbour = neighbour
                
        assert merge_to_neighbour is not None, (x, y)
        # assert min_neighbour_n_images > 0, (x, y)
        
        assignment_merged[merge_to_neighbour[1], merge_to_neighbour[0]] = torch.logical_or(
            assignment_merged[y, x],
            assignment_merged[merge_to_neighbour[1], merge_to_neighbour[0]],
        )
        assignment_merged[y, x].fill_(False)
        
        print("({}, {}) -> ({}, {})".format(x + scene.scene_bounding_box.origin_partition_offset[0], y + scene.scene_bounding_box.origin_partition_offset[1], merge_to_neighbour[0] + scene.scene_bounding_box.origin_partition_offset[0], merge_to_neighbour[1] + scene.scene_bounding_box.origin_partition_offset[1]))
        # break
    # break
assignment_merged.sum(-1)

In [ ]:
# assign with enlarged partition
scene_config.location_based_enlarge = 0.2  # the merging requires overlap
scene.camera_center_based_partition_assignment().sum(-1).reshape(scene.scene_bounding_box.n_partitions.tolist()[::-1])

In [ ]:
# mask out those partitions merged into others
masked_assignment = torch.logical_and(scene.is_camera_in_partition, assignment_merged.sum(-1).reshape((-1, 1)) > 0)
masked_assignment.sum(-1).reshape(scene.scene_bounding_box.n_partitions.tolist()[::-1])

In [ ]:
# merge the enlarged and merged assignments
final_assignment = torch.logical_or(masked_assignment, assignment_merged.reshape(masked_assignment.shape))
final_assignment.sum(-1).reshape(scene.scene_bounding_box.n_partitions.tolist()[::-1])

In [ ]:
from matplotlib.pyplot import cm
import matplotlib.patches as mpatches
import random

plt.close()
fig, ax = plt.subplots()
scene.set_plot_ax_limit(ax)
ax.set_aspect('equal', adjustable='box')
colors = list(iter(cm.rainbow(np.linspace(0, 1, len(scene.partition_coordinates)))))
random.shuffle(colors)
color_iter = iter(colors)

annotate_position_x = 0.125
annotate_position_y = 0.25
annotate_font_size = 5

idx = 0
local_idx = 0
for partition_id, partition_xy in scene.partition_coordinates:
    try:
        assigned_camera_mask = scene.is_camera_in_partition[idx]
        color=next(color_iter)
        
        if assigned_camera_mask.sum() == 0:
            continue
            
        assigned_camera_centers = camera_centers[assigned_camera_mask.numpy()]
                        
        ax.scatter(
            assigned_camera_centers[:, 0], 
            assigned_camera_centers[:, 1],
            s=0.2,
            c=color[None, :] * np.asarray([[1., 1., 1., 0.1]]),
        )

        ax.add_artist(mpatches.Rectangle(
            (partition_xy[0], partition_xy[1]),
            scene.scene_config.partition_size,
            scene.scene_config.partition_size,
            fill=False,
            color=color,
        ))
        ax.annotate(
            "#{}\n({}, {})".format(local_idx, partition_id[0], partition_id[1]),
            xy=(
                partition_xy[0] + annotate_position_x * scene.scene_config.partition_size,
                partition_xy[1] + annotate_position_y * scene.scene_config.partition_size,
            ),
            fontsize=annotate_font_size,
        )
        local_idx += 1
    finally:
        idx += 1

plt.savefig(os.path.join(output_path, "sfm_partitions.png"), dpi=600)
plt.show(fig)

In [ ]:
# make sure all images are covered
assert (final_assignment.sum(dim=0) > 0).all()

In [ ]:
np.save(os.path.join(output_path, "sfm_partitions.npy"), final_assignment.numpy())

In [ ]:
final_assignment.numpy().shape

In [ ]:
image_name_array = np.asarray(image_names)

In [ ]:
image_list_output_path = os.path.join(output_path, "sfm_partition_image_lists")
os.makedirs(image_list_output_path, exist_ok=True)

for i in os.scandir(image_list_output_path):
    if not i.is_dir(follow_symlinks=False):
        os.unlink(i.path)

local_idx = 0
for partition in final_assignment:
    if partition.sum() == 0:
        continue
        
    partition_image_name_array = image_name_array[partition]
    with open(os.path.join(image_list_output_path, "{:02d}.txt".format(local_idx)), "w") as f:
        for i in partition_image_name_array:
            f.write(i)
            f.write("\n")
    
    local_idx += 1
n_sfm_partitions = local_idx
image_list_output_path

### 7.2.2. Mapping
Build colmap with cuDSS to enable mapper with GPU


In [ ]:
sfm_partition_dir = os.path.join(output_path, "sfm_partitions")
os.makedirs(sfm_partition_dir, exist_ok=True)

remove_abs_path = len(os.path.dirname(image_dir)) + 1

print_command(["chmod", "400", colmap_db_path[remove_abs_path:]])  # key to run multiple mapper in parallel
print()
for partition_image_list_file in os.scandir(os.path.join(output_path, "sfm_partition_image_lists")):
    partition_idx = int(partition_image_list_file.name.rsplit(".")[0])
    sfm_partition_sparse_dir = os.path.join(sfm_partition_dir, "{:02d}".format(partition_idx))
    os.makedirs(sfm_partition_sparse_dir, exist_ok=True)
    print_command(["mkdir", "-p", sfm_partition_sparse_dir[remove_abs_path:]])
    print(get_command([
        "srun",
        "--gpus=1",
        "--nodes=1",
        "--ntasks=1",
        "--exclusive",
        "colmap",
        "mapper",
        "--image_list_path={}".format(partition_image_list_file.path[remove_abs_path:]),
        "--Mapper.ba_use_gpu=1",
        # "--Mapper.min_num_matches=32",
        "--database_path={}".format(colmap_db_path[remove_abs_path:]),
        "--image_path={}".format(image_dir[remove_abs_path:]),
        "--output_path={}".format(sfm_partition_sparse_dir[remove_abs_path:]),        
    ]) + " &\n")
print("wait")

### 7.2.3. Remove cameras with large error values compared to GPS
This section can be run independently

In [ ]:
from tqdm.auto import tqdm
from internal.utils import sfm_outlier_detection

In [ ]:
sfm_partition_load_dir = os.path.expanduser("~/dataset/dbl/parsed_from_exif-sift_matching/sfm_partitions")
# outlier_removed_partition_output_dir = os.path.expanduser("~/dataset/dbl/parsed_from_exif-sift_matching/sfm_partitions-outliers_removed")
outlier_removed_partition_output_dir = "{}-outliers_removed".format(sfm_partition_load_dir)

In [ ]:
def find_largest_sparse_model(path: str):
    # find model with the most images
    sfm_partition_sparse_model_dir = None
    sfm_partition_cameras_bin_max_size = -1
    for sparse_model_idx_dir in os.scandir(path):
        if not sparse_model_idx_dir.is_dir():
            continue
        images_bin_path = os.path.join(sparse_model_idx_dir.path, "images.bin")
        if not os.path.exists(images_bin_path):
            continue
        images_bin_size = os.stat(images_bin_path).st_size
        if images_bin_size > sfm_partition_cameras_bin_max_size:
            sfm_partition_cameras_bin_max_size = images_bin_size
            sfm_partition_sparse_model_dir = sparse_model_idx_dir.path
    assert sfm_partition_sparse_model_dir is not None
    return sfm_partition_sparse_model_dir

In [ ]:
for sfm_partition_dir in tqdm(sorted(list(os.scandir(sfm_partition_load_dir)), key=lambda i: i.name)):
    if not sfm_partition_dir.is_dir():
        continue
    sfm_partition_idx_str = sfm_partition_dir.name
    sfm_partition_idx = int(sfm_partition_idx_str)

    largest_sparse_model_dir = find_largest_sparse_model(sfm_partition_dir.path)
    
    colmap_images = sfm_outlier_detection.load(prior_poses, largest_sparse_model_dir)
    sfm_outlier_detection.filter(
        colmap_images,
        output_path=os.path.join(outlier_removed_partition_output_dir, sfm_partition_idx_str),
        min_acceptable_error_limit=6.,
        max_acceptable_error_limit=64.,
    )()

### 7.2.4. Merge partitions

Define the merge order

TODO: calculate the order based on overlapping

TODO: BA for every N partitions merged

In [ ]:
# Partition IDs; DFS
merge_order = [
    [1, 2, 0],
    [3, 4, 5],
    [6, 7, 8, 9],
    [10, 11, 12, 13],
    [14, 15],
]
merged_partition_output_path = "{}-merged".format(sfm_partition_load_dir)
merged_partition_output_path

In [ ]:
from typing import Union, List
import subprocess

# validate the merge order
partition_id_appear_counters = {}
def validate_merge_order(m):
    if isinstance(m, List) and len(m) == 1:
        m = m[0]
    if isinstance(m, int):
        assert m not in partition_id_appear_counters, "partition #{} appears twice".format(m)
        partition_id_appear_counters[m] = 1
        return 1

    total = 0
    for i in m:
        total += validate_merge_order(i)
    return total

n_partitions_to_merge = validate_merge_order(merge_order)
merged_counter = 1

# DFS merging
def merge_partitions(partition_ids: Union[int, List]) -> str:
    if isinstance(partition_ids, List) and len(partition_ids) == 1:
        partition_ids = partition_ids[0]
    if isinstance(partition_ids, int):
        return os.path.join(outlier_removed_partition_output_dir, "{:02d}".format(partition_ids))
    
    sub_partitions = []
    for i in partition_ids:
        sub_partitions.append(merge_partitions(i))

    merged = sub_partitions[0]
    for i in sub_partitions[1:]:
        output_name = "{}-{}".format(os.path.basename(merged), os.path.basename(i))
        output_path = os.path.join(merged_partition_output_path, output_name)
        os.makedirs(output_path, exist_ok=True)
        if os.path.exists(os.path.join(output_path, "images.bin")):
            print("skip {}".format(output_name))
        else:
            args = [
                "colmap",
                "model_merger",
                "--input_path1={}".format(merged),
                "--input_path2={}".format(i),
                "--output_path={}".format(output_path),
            ]
            print_command(args)
            assert subprocess.call(args) == 0
        global merged_counter
        merged_counter += 1
        print("{} of {} merged".format(merged_counter, n_partitions_to_merge))
        merged = output_path
    
    return merged

    
final_merged_sparse_model = merge_partitions(merge_order)
final_merged_sparse_model_alias = os.path.join(os.path.dirname(final_merged_sparse_model), "final")
if os.path.exists(final_merged_sparse_model_alias):
    os.unlink(final_merged_sparse_model_alias)
os.symlink(final_merged_sparse_model, final_merged_sparse_model_alias)
final_merged_sparse_model_alias

### 7.2.5. point_triangulator and bundle_adjuster

In [ ]:
tri_output_path = "{}-retriangulated".format(final_merged_sparse_model_alias)
os.makedirs(tri_output_path, exist_ok=True)
print_command([
    "colmap",
    "point_triangulator",
    "--database_path={}".format(colmap_db_path),
    "--image_path={}".format(image_dir),
    "--input_path={}".format(final_merged_sparse_model_alias),
    "--output_path={}".format(tri_output_path),
    "--Mapper.ba_use_gpu=1",
])

In [ ]:
# Turn to `gps_based_sfm_outlier_detection.ipynb`, or BA directly.

ba_output_path = "{}-bundle_adjusted".format(tri_output_path)
os.makedirs(ba_output_path, exist_ok=True)
print_command([
    "colmap",
    "bundle_adjuster",
    "--input_path={}".format(tri_output_path),
    "--output_path={}".format(ba_output_path),
    "--BundleAdjustment.use_gpu=1",
])